In [5]:
from dotenv import load_dotenv
from langchain.tools import tool
from typing import Dict, Any, Callable
from tavily import TavilyClient
from langchain_community.utilities import SQLDatabase
from dataclasses import dataclass
from langchain.messages import HumanMessage
from langchain.chat_models import init_chat_model
from langchain.agents import create_agent
from langchain.agents.middleware import wrap_model_call, ModelRequest, ModelResponse
from typing import Callable


In [6]:
load_dotenv()

True

In [9]:
tavily_client = TavilyClient()

db = SQLDatabase.from_uri("sqlite:///resources/Chinook.db")


@tool
def web_search(query: str) -> Dict[str, Any]:

    """Search the web for information"""

    return tavily_client.search(query)

@tool
def sql_query(query: str) -> str:

    """Obtain information from the database using SQL queries"""

    try:
        return db.run(query)
    except Exception as e:
        return f"Error: {e}"

In [7]:
@dataclass
class UserRole:
    user_role: str = "external"

In [ ]:
@wrap_model_call
def dynamic_tool_call(request: ModelRequest, handler: Callable[[ModelRequest], ModelResponse]) -> ModelResponse:

    """Dynamically call tools based on the runtime context"""

    user_role = request.runtime.context.user_role

    if user_role == "internal":
        pass # internal users get access to all tools
    else:
        tools = [web_search] # external users only get access web search
        request = request.override(tools=tools)

    return handler(request)

In [11]:
model = init_chat_model(model="gemini-2.5-flash", model_provider="google_genai")

In [12]:
agent = create_agent(
    model = model,
    tools=[web_search, sql_query],
    middleware=[dynamic_tool_call],
    context_schema=UserRole
)

In [18]:
response = agent.invoke(
    {"messages": [HumanMessage(content="How many artists are in the database?")]},
    context={"user_role": "external"}
)


In [19]:
print(response["messages"][-1].content[0]["text"])

I can't answer that question. My capabilities do not include accessing or querying a database of artists.


In [20]:
response = agent.invoke(
    {"messages": [HumanMessage(content="How many artists are in the database?")]},
    context={"user_role": "internal"}
)


In [21]:
print(response["messages"][-1].content[0]["text"])

There are 275 artists in the database.
